In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt 
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [2]:
class MLP(nn.Module):
    def __init__(self, layer_dims):
        super().__init__()
        self.layyers = nn.ModuleList()
        for i in range(len(layer_dims) - 1):
            self.layyers.append(nn.Linear(layer_dims[i], layer_dims[i + 1]))
        self.activation = nn.Tanh()

    def forward(self, x):
        for layer in self.layyers[:-1]:
            x = self.activation(layer(x))
        x = self.layyers[-1](x)
        return x


In [4]:
class PINN_Burgers(nn.Module):
    def __init__(self, layer_dims, true_nu=None):
        super().__init__()
        self.network = MLP(layer_dims)
        
        # 将 nu 定义为一个可学习的参数
        # 我们用一个猜测值进行初始化，例如 0.1
        # 如果提供了 true_nu，则使用它（用于调试/测试正向问题）
        initial_nu = 0.1 if true_nu is None else true_nu
        self.nu = nn.Parameter(torch.tensor([initial_nu], dtype=torch.float32, requires_grad=True))
    
    def forward(self, x, t):
        # 拼接 x 和 t 以创建网络输入
        inputs = torch.cat([x, t], dim=1)
        return self.network(inputs)
    
    def compute_pde_residual(self, x, t):
        # 为输入设置 requires_grad=True 以计算导数
        x.requires_grad_(True)
        t.requires_grad_(True)
        
        u = self.forward(x, t)
        
        # 使用自动微分计算导数
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        
        # 伯格斯方程残差
        residual = u_t + u * u_x - self.nu * u_xx
        return residual


In [ ]:
# -- 2. 主训练脚本 --
if __name__ == '__main__':
    # 设备设置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
 # --- 数据加载和准备 ---
    data = np.load('burgers_shock_solution.npz')
    x_data = torch.tensor(data['x'], dtype=torch.float32) # 形状: [nx, 1]
    t_data = torch.tensor(data['t'], dtype=torch.float32) # 形状: [nt, 1]
    u_solution = torch.tensor(data['u'], dtype=torch.float32) # 形状: [nt, nx]

    # 创建用于训练的坐标网格
    # 我们将在所有数据点上进行训练
    T, X = torch.meshgrid(t_data.squeeze(), x_data.squeeze(), indexing='ij')
    
   # 准备训练数据三元组 (x, t, u)
    x_train = X.reshape(-1, 1)
    t_train = T.reshape(-1, 1)
    u_train = u_solution.reshape(-1, 1)

# 将所有训练数据移动到选定的设备
    x_train = x_train.to(device)
    t_train = t_train.to(device)
    u_train = u_train.to(device)


  # --- 使用 TensorDataset 和 DataLoader 创建小批量数据 ---
    # 定义批次大小，您可以根据您的GPU显存进行调整
    # batch_size = 2048
    # train_dataset = TensorDataset(x_train, t_train, u_train)
    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# --- 模型、损失和优化器设置 ---
# 对于逆向问题，我们不提供真实的 nu
# pinn_model.parameters() 是一个“参数收集器,返回一个包含所有找到的参数的集合
pinn_model = PINN_Burgers(layer_dims=[2, 20, 1]).to(device)
optimizer = torch.optim.Adam(pinn_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

Using device: cuda


In [7]:
 # --- 训练循环 ---
epochs = 20000
# 定义一个裁剪阈值
clip_value = 1.0 
for epoch in range(epochs):
    
    optimizer.zero_grad()

    # 1. 数据损失
    u_pred = pinn_model(x_train, t_train)
    data_loss = loss_fn(u_pred, u_train)
# 2. 物理损失
    # 对于物理损失，我们可以使用相同的点或采样新的点。
    # 这里为了简单起见，我们使用相同的训练点。
    pde_residual = pinn_model.compute_pde_residual(x_train, t_train)
    physics_loss = loss_fn(pde_residual, torch.zeros_like(pde_residual))

    # 或者 0.01，这是一个需要调整的超参数
    lambda_physics = 5
    physics_loss = lambda_physics * physics_loss
    # 总损失（您可以添加一个权重，例如 total_loss = data_loss + 0.1 * physics_loss）
    total_loss = data_loss + physics_loss

    total_loss.backward()

    # 在更新之前进行梯度裁剪
    torch.nn.utils.clip_grad_norm_(pinn_model.parameters(), clip_value)
    optimizer.step()

    if (epoch + 1) % 1000 == 0:
        # 秘密答案是 0.07。让我们看看我们的猜测有多接近！
        print(f"Epoch [{epoch+1}/{epochs}], Total Loss: {total_loss.item():.4f}, "
                f"Data Loss: {data_loss.item():.4f}, Physics Loss: {physics_loss.item():.4f}, "
                f"Predicted nu: {pinn_model.nu.item():.4f}")
print("\nTraining finished!")
print(f"The final predicted viscosity nu is: {pinn_model.nu.item():.5f}")
print(f"(The true value was 0.07)")   

    


Epoch [1000/20000], Total Loss: 0.0127, Data Loss: 0.0112, Physics Loss: 0.0015, Predicted nu: 0.0515
Epoch [2000/20000], Total Loss: 0.0126, Data Loss: 0.0111, Physics Loss: 0.0015, Predicted nu: 0.0512
Epoch [3000/20000], Total Loss: 0.0126, Data Loss: 0.0111, Physics Loss: 0.0015, Predicted nu: 0.0510
Epoch [4000/20000], Total Loss: 0.0126, Data Loss: 0.0111, Physics Loss: 0.0016, Predicted nu: 0.0509
Epoch [5000/20000], Total Loss: 0.0126, Data Loss: 0.0110, Physics Loss: 0.0015, Predicted nu: 0.0508
Epoch [6000/20000], Total Loss: 0.0126, Data Loss: 0.0110, Physics Loss: 0.0015, Predicted nu: 0.0508
Epoch [7000/20000], Total Loss: 0.0125, Data Loss: 0.0110, Physics Loss: 0.0015, Predicted nu: 0.0508
Epoch [8000/20000], Total Loss: 0.0125, Data Loss: 0.0110, Physics Loss: 0.0015, Predicted nu: 0.0508
Epoch [9000/20000], Total Loss: 0.0125, Data Loss: 0.0110, Physics Loss: 0.0015, Predicted nu: 0.0508
Epoch [10000/20000], Total Loss: 0.0125, Data Loss: 0.0110, Physics Loss: 0.0015, 